<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Data Science in IoT<br/>Asignatura Data preprocessing</center>

# Challenge S7: Clasificación de sentimientos en tweets

## Objetivos:

El objetivo de este challenge es enfrentarse a un problema de clasificación de texto real: tweets descargados sobre las elecciones de EEUU en 2016, centrándonos en el preprocesamiento, que en este caso es crucial en un problema en el que el protagonista es el texto.

## Inicialización del entorno:

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import random

random.seed(1234)
pd.set_option('display.max_colwidth', 175) # incrementamos anchura de output

In [2]:
df_train = pd.read_csv('./data/gop_tweets_train_psn.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8414 entries, 0 to 8413
Data columns (total 2 columns):
sentiment    8414 non-null object
text         8414 non-null object
dtypes: object(2)
memory usage: 131.5+ KB


In [3]:
df_test = pd.read_csv('./data/gop_tweets_test_psn.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2153 entries, 0 to 2152
Data columns (total 2 columns):
sentiment    2153 non-null object
text         2153 non-null object
dtypes: object(2)
memory usage: 33.7+ KB


Damos un vistazo a los textos que aparecen en el dataset.

In [4]:
df_train.head()

,sentiment,text
0,Neutral,RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate
1,Positive,RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…
2,Neutral,RT @TJMShow: No mention of Tamir Rice and the #GOPDebate was held in Cleveland? Wow.
3,Positive,RT @RobGeorge: That Carly Fiorina is trending -- hours after HER debate -- above any of the men in just-completed #GOPdebate says she's on …
4,Positive,RT @DanScavino: #GOPDebate w/ @realDonaldTrump delivered the highest ratings in the history of presidential debates. #Trump2016 http://t.co…


In [5]:
df_test.head()

,sentiment,text
0,Positive,"RT @GregAbbott_TX: @TedCruz: ""On my first day I will rescind every illegal executive action taken by Barack Obama."" #GOPDebate @FoxNews"
1,Neutral,Going on #MSNBC Live with @ThomasARoberts around 2 PM ET. #GOPDebate
2,Positive,RT @WayneDupreeShow: Just woke up to tweet this out #GOPDebate \r\n\r\nBest line of the night via @GovMikeHuckabee http://t.co/6OV5hxHIcV
3,Negative,reason comment is funny 'in case you're ignorant' is the #gop #tcot are the reason the government isn't working for the people #gopdebate
4,Positive,"RT @ChuckNellis: Cruz has class &amp; truth, that gets my vote! #GOPDebate"


## ¿Qué número de tweets tenemos en cada dataset? ¿Cuántos de cada clase?

In [8]:
df_train["sentiment"].value_counts(normalize=True)

Negative    0.579035
Neutral     0.256953
Positive    0.164012
Name: sentiment, dtype: float64

## ¿Qué dificultades crees que presenta la diferencia entre la cantidad de clases?

In [9]:
df_test["sentiment"].value_counts(normalize=True)

Negative    0.583837
Neutral     0.263353
Positive    0.152810
Name: sentiment, dtype: float64

## Clasificador de tweets

Vamos a construir nuestro propio clasificador sobre los tweets que hemos cargado. Los pasos que seguiremos serán:

* Preprocesar ambos conjuntos de tweets por separado.
* Entrenar clasificador con tweets de entrenamiento.
* Evaluar con tweets de test.

Primero, haremos una prueba para observar el rendimiento de nuestro clasificador sin ningún tipo de preprocesamiento.

In [10]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

Seguimos creando el clasificador y entrenando con el conjunto de entrenamiento de los tweets.

In [11]:
# creamos nuestro pipeline con vectorizador (bag of words) y clasificador
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('clf', LinearSVC(max_iter=1200))])

# entrenamos el clasificador
text_clf.fit(df_train['text'], df_train['sentiment'])

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1200,
                           multi_class='ovr', penalty='l2', random_state=None,
                           tol=0.000

Realizamos las predicciones con el conjunto de test y obtenemos la métrica de evaluación.

In [12]:
predicted = text_clf.predict(df_test['text'])
np.mean(predicted == df_test['sentiment'])

0.5973060845332094

Parece que hemos obtenido un resultado un poco peor... Ahora es cuando entra en juego el preprocesamiento del texto.

Ahora vamos a preprocesar el texto y construir un clasificador. Para ello, se ha creado esta función que nos automatiza el preprocesamiento. Deberás añadir el código que creas necesario y ejecutar la función para obtener el texto preprocesado. 

**Puedes definir todas las funciones de preprocesado que creas necesario, pero ten en cuenta varios detalles:**
* La funcion `preprocesar_texto` es llamada desde un `apply`. Esto significa que la función recibe un tweet y devuelve un tweet preprocesado con cada llamada. 
* Si el preprocesado es muy agresivo, un tweet puede quedarse fácilmente vacío, en esos casos el clasificador no podrá trabajar con él y escupirá la clase mayoritaria para ese tweet.
* Algunas técnicas no tienen sentido si se aplican después de otras dentro del pipeline. Por ejemplo, no tiene sentido aplicar POS después de realizar Stemming.
* Count Vectorizer necesita DOCUMENTOS y al trabajar con texto muchas veces es más sencillo hacer uso de TOKENS. Esto significa que tendremos que pasar cada tweet preprocesado en una cadena de texto. La función `" ".join(palabras)` que vimos en el worksheet es tu aliada.
* Si tu preprocesamiento va a ser intensivo, quizás es buena idea crear varias funciones que sean llamadas desde `preprocesar_texto`.

In [113]:
# A minúsculas
def to_lower(text):
    return text.lower()

text_lower = to_lower(df_train['text'][1])
text_lower

"rt @scottwalker: didn't catch the full #gopdebate last night. here are some of scott's best lines in 90 seconds. #walker16 http://t.co/zsff…"

In [114]:
## Tokenizamos
from nltk.tokenize import TweetTokenizer

def get_tokens(text):
    return TweetTokenizer(strip_handles=True).tokenize(text)

text_tokens = get_tokens(text_lower)
print(text_tokens)

['rt', ':', "didn't", 'catch', 'the', 'full', '#gopdebate', 'last', 'night', '.', 'here', 'are', 'some', 'of', "scott's", 'best', 'lines', 'in', '90', 'seconds', '.', '#walker16', 'http://t.co/zsff…']


In [115]:
## Limpiamos stopwords
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords.extend([
    '``','/','=','(',')','\'','...','-',
    ',', '.', ';', ':', '?', '¿', '!', '¡',
    '%','🇸','🇺','@','http',
    '…','\"','&','’','“','”','#'
])

def clean_stopwords(tokens, stopwords):
    clean_tokens = []
    for token in tokens:
        if token in stopwords:
            continue
        clean_tokens.append(token)
    return clean_tokens

clean_tokens = clean_stopwords(text_tokens, stopwords)
print(clean_tokens)

['rt', 'catch', 'full', '#gopdebate', 'last', 'night', "scott's", 'best', 'lines', '90', 'seconds', '#walker16', 'http://t.co/zsff…']


In [116]:
# limpiamos más basura

def clean_twitter_conventions(tokens):
    clean_tokens = []
    for token in tokens:
        if token.find('@') >= 0 or token.find('#') >= 0:
            continue
            
        if token.find('http') >= 0:
            continue
        
        if token == 'rt':
            continue
        
        clean_tokens.append(token)
    return clean_tokens

clean_tokens = clean_twitter_conventions(clean_tokens)
print(clean_tokens)

['catch', 'full', 'last', 'night', "scott's", 'best', 'lines', '90', 'seconds']


In [117]:
# stemming
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

def get_stemmed_text(clean_tokens, stemmer):
    tokens_stem = []
    for token in clean_tokens:
        tokens_stem.append(stemmer.stem(token))
    
    return tokens_stem

clean_tokens = get_stemmed_text(clean_tokens, stemmer)
print(clean_tokens)

['catch', 'full', 'last', 'night', 'scott', 'best', 'line', '90', 'second']


In [120]:
# filtrar POS
from nltk import pos_tag
selected_pos = ['JJ', 'NN', 'NNS']

def filter_pos(clean_tokens, selected_tokens, pos_tag):
    tokens_filtered = []
    pos = pos_tag(clean_tokens)
    for p in pos:
        if p[1] in selected_pos:
            tokens_filtered.append(p[0])
            
    return tokens_filtered

clean_tokens = filter_pos(clean_tokens, selected_tokens, pos_tag)
print(clean_tokens)

['catch', 'full', 'last', 'night', 'scott', 'line', 'second']


In [127]:
def preprocesar_texto(texto):
    """ Función para preprocesamiento de texto.
    Args:
        texto: cadena de texto a preprocesar.
    Returns:
        mismo texto preprocesado.
    """
    texto_filtrado = ""
    texto_filtrado = to_lower(texto)
    texto_filtrado = get_tokens(texto_filtrado)
    texto_filtrado = clean_stopwords(texto_filtrado, stopwords)
    texto_filtrado = clean_twitter_conventions(texto_filtrado)
    
    # técnicas que con una implementación sencilla no logran mejorar resultados
    # texto_filtrado = filter_pos(texto_filtrado, selected_tokens, pos_tag)
    # texto_filtrado = get_stemmed_text(texto_filtrado, stemmer)
    
    
    texto_filtrado = ' '.join(texto_filtrado)
    return texto_filtrado

Aplicamos el preprocesamiento a los dos conjuntos de tweets y lo guardamos en la columna `prep`.

In [128]:
df_train['prep'] = df_train['text'].apply(preprocesar_texto)
df_test['prep'] = df_test['text'].apply(preprocesar_texto)

Antes de reentrenar nuestro clasificador, siempre es recomendable dar un vistazo al resultado del texto. Podemos observar frecuencia de palabras o simplemente generar un WordCloud.

In [129]:
df_train['prep'][0:10]

0           everyone feel climate change question last night exactly
1                catch full last night scott's best lines 90 seconds
2                              mention tamir rice held cleveland wow
3        carly fiorina trending hours debate men just-completed says
4           w delivered highest ratings history presidential debates
5                          liked happy heard going moderator anymore
6    deer headlights ben carson may brain surgeon performed lobotomy
7                                         last night's debate proved
8                                               fairness owns phrase
9                                    reading family's comments great
Name: prep, dtype: object

Ahora sí, vamos a reentrenar el clasificador y evaluar de nuevo. Puedes hacer uso del mismo pipeline anterior o simplemente crear uno nuevo con más etapas.

In [130]:
# Nuevo pipeline si es que es necesario. Por defecto usamos el anterior.
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

new_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer(use_idf = True, smooth_idf=True)),
                    ('clf', LinearSVC())])

In [131]:
new_clf.fit(df_train['text'], df_train['sentiment'])

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                     

In [132]:
# Predicción haciendo uso del pipeline correspondiente. ¡Recuerda cambiarlo si has creado uno nuevo!
predicted = new_clf.predict(df_test['prep'])
np.mean(predicted == df_test['sentiment'])

0.6228518346493265

* ¿Qué opinas de los resultados obtenidos?
* ¿Por qué crees que ha sucedido esto?

¿Podrías explicarme con tus palabras en qué consiste Tf-Idf? ¿Cual es la idea intuitiva de aplicar esta transformación a la matriz?

Tf-Idf es una técnica que se aplica a la matriz de documentos-términos. Significa "term frequency - inverse document frequency" y es una técnica que especifica cómo de importante es un término dentro de una colección de documentos. A su vez, está formada por dos técnicas diferentes que son las siguientes:

* Term frequency: que recompensa aquellas palabras que aparecen a menudo en un documento.
* Inverse document frequency: que penaliza aquellas palabras que aparecen a menudo en una colección de documentos.

Intuitivamente, buscará los términos más importantes para identificar un documento dentro de una colección de documentos. Hará que aumente el valor ponderado del término dentro de un documento, pero disminuirá su valor si este término es común en muchos documentos. A mayor importancia, más alto es el valor en el resultado final.